# Posterior Decoding
Implementation of the Posterior Decoding algorithm.



## Python Imports

In [41]:
import numpy as np

## Encode sequence as integers (index values)

In [42]:
def encode( sequence, symbols):
    
    enc = [0] * len(sequence)
    
    for i in range(len(sequence)):
        enc[i] = symbols.find(sequence[i])
    
    return(enc)

## Parameters

In [43]:
states = 2

symbols = "123456"

#input_sequence = "566611234"
#input_sequence = "31245366664"
input_sequence = "34512331245366664666563266"

input_encode = encode( input_sequence, symbols)

initial_prob = [1.0/states, 1.0/states]

transition_matrix = np.asarray([0.95, 0.05, 0.1, 0.9]).reshape(2,2)

fair_prob = [1.0/6, 1./6, 1./6, 1./6, 1./6, 1./6]
loaded_prob = [1./10, 1./10, 1./10, 1./10, 1./10, 5./10] 
emission_probs = [fair_prob, loaded_prob]


## Forward Loop
### Remember that we here do NOT work in log space

In [44]:
def initialize_forward(input_encode, states, initial_prob, emission_probs):
    
    alpha = np.zeros(shape=(states, len(input_encode)))
        
    for i in range(0, states): 
        
        alpha[i][0] = initial_prob[i]*emission_probs[i][input_encode[0]]
        
    return alpha


In [45]:
alpha = initialize_forward(input_encode, states, initial_prob, emission_probs)

# main loop
for i in range(1, len(input_encode)):
    
    for j in range(0, states):

        _sum = 0
        
        for k in range(0, states):
            
            _sum += alpha[k][i-1]*transition_matrix[k][j]   #XX           
         
        # store prob
        alpha[j][i] = emission_probs[j][input_encode[i]] * _sum


## Backward Loop

In [46]:
def initialize_backward(input_encode, states):
    
    #beta = np.zeros(shape=(states, len(input_encode), dtype=float))
    beta = np.zeros(shape=(states, len(input_encode)))
        
    for i in range(0, states):
  
        beta[i][-1] = 1
        
    return beta

In [47]:
beta = initialize_backward(input_encode, states)

# main loop
for i in range(len(input_encode)-2, -1, -1):
    
    for j in range(0, states):

        _sum = 0

        for k in range(0, states):
            
            _sum +=  transition_matrix[j][k] * beta[k][i+1] * emission_probs[k][input_encode[i+1]] #
        
         #store prob
        beta[j][i] = _sum  


## Posterior Loop
 

In [48]:
# posterior = f * b / p_x

posterior = np.zeros(shape=(len(input_encode)), dtype=float)

p_state = 0

p_x = 0
for j in range(0, states):
    p_x += alpha[j][-1]

print ("Log(Px):", np.log(p_x))

for i in range(0, len(input_encode)):
        
    posterior[i] = (alpha[p_state][i]*beta[p_state][i])/p_x # p = (f_i * b_i)/p_x

    print ("Posterior", i, input_sequence[i], input_encode[i], np.log(alpha[p_state, i]), np.log(beta[p_state, i]), posterior[i])

Log(Px): -41.70474157060052
Posterior 0 3 2 -2.4849066497880004 -39.35940929634274 0.8697283342927764
Posterior 1 4 3 -4.266715788162887 -37.52445740365702 0.917198263539344
Posterior 2 5 4 -6.073538637630933 -35.68995230080138 0.9429430714098719
Posterior 3 1 0 -7.893430617460898 -33.85625161703387 0.9560542086837365
Posterior 4 2 1 -9.720352401473978 -32.0240017844215 0.9611617063864735
Posterior 5 3 2 -11.551114466753617 -30.19437700488653 0.9600692122362451
Posterior 6 3 2 -13.383992144478592 -28.369529118205953 0.9523909258478439
Posterior 7 1 0 -15.218040712005948 -26.55346547351473 0.9354153582437182
Posterior 8 2 1 -17.052738967590443 -24.753871660818092 0.903147804079745
Posterior 9 4 3 -18.887798224281546 -22.986317024421123 0.844193387856058
Posterior 10 5 4 -20.723058230083396 -21.28585650141364 0.7377331088630769
Posterior 11 3 2 -22.55842991926611 -19.75113321212816 0.5461718764211234
Posterior 12 6 5 -24.393863756721874 -18.911047396484218 0.20186228276000984
Posterior 1